# Solving Unconstrained Nonlinear Programming Models


In [1]:
# load libraries
import numpy as np
import scipy.optimize as opt

### Newton-Conjugate-Gradient algorithm
Newton-Conjugate Gradient algorithm is a modified Newton’s method and uses a conjugate gradient algorithm to (approximately) invert the local Hessian. Newton’s method is based on fitting the function locally to a quadratic form:

$$f\left(\mathbf{x}\right)\approx f\left(\mathbf{x}_{0}\right)+\nabla f\left(\mathbf{x}_{0}\right)^{T} \left(\mathbf{x}-\mathbf{x}_{0}\right)+\frac{1}{2}\left(\mathbf{x}-\mathbf{x}_{0}\right)^{T}\mathbf{H}\left(\mathbf{x}_{0}\right)\left(\mathbf{x}-\mathbf{x}_{0}\right),$$

where $\mathbf{H}\left(\mathbf{x}_{0}\right)$ is a matrix of second-derivatives (the Hessian). If the Hessian is positive definite then the local minimum of this function can be found by setting the gradient of the quadratic form to zero, resulting in

$$\mathbf{x}_{\textrm{opt}}=\mathbf{x}_{0}-\mathbf{H}^{-1}\nabla f.$$

The inverse of the Hessian is evaluated using the conjugate-gradient method. An example of employing this method to minimizing the Rosenbrock function is given below. To take full advantage of the Newton-CG method, a function which computes the Hessian must be provided. The Hessian matrix itself does not need to be constructed, only a vector which is the product of the Hessian with an arbitrary vector needs to be available to the minimization routine. As a result, the user can provide either a function to compute the Hessian matrix, or a function to compute the product of the Hessian with an arbitrary vector.

The Hessian of the Rosenbrock function is

\begin{align*} H_{ij}=\frac{\partial^{2}f}{\partial x_{i}\partial x_{j}} =& \left(202+1200x_{i}^{2}-400x_{i+1}\right)\delta_{i,j}-400x_{i}\delta_{i+1,j}-400x_{i-1}\delta_{i-1,j},\end{align*}

if $i\in\left[2,N-1\right]$ and $j\in\left[2,N-1\right]$ defining the $N \times N$ matrix. Other non-zero entries of the matrix are

\begin{align*} \frac{\partial^{2}f}{\partial x_{1}^{2}} = & 1200x_{1}^{2}-400x_{2}+2,\\ \frac{\partial^{2}f}{\partial x_{1}\partial x_{2}}=\frac{\partial^{2}f}{\partial x_{2}\partial x_{1}} =& -400x_{1},\\ \frac{\partial^{2}f}{\partial x_{N-1}\partial x_{N}}=\frac{\partial^{2}f}{\partial x_{N}\partial x_{N-1}} =& -400x_{N-1},\\ \frac{\partial^{2}f}{\partial x_{N}^{2}} = & 200.\end{align*}

For example, the Hessian when $N = 5$ is

\begin{split}\mathbf{H}=\begin{bmatrix} 1200x_{1}^{2}-400x_{2}+2 & -400x_{1} & 0 & 0 & 0\\ -400x_{1} & 202+1200x_{2}^{2}-400x_{3} & -400x_{2} & 0 & 0\\ 0 & -400x_{2} & 202+1200x_{3}^{2}-400x_{4} & -400x_{3} & 0\\ 0 &  & -400x_{3} & 202+1200x_{4}^{2}-400x_{5} & -400x_{4}\\ 0 & 0 & 0 & -400x_{4} & 200\end{bmatrix}.\end{split}

The code which computes this Hessian along with the code to minimize the function using Newton-CG method is shown in the following example:

In [2]:
def Rosenbrock(x):
    return np.sum(100.0 * (x[1:] - x[:-1]**2)**2 + (1 - x[:-1])**2)

In [3]:
def Rosenbrock_derivative(x):
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    derivative = np.zeros_like(x)
    derivative[1:-1] = 200 * (xm - xm_m1**2) - 400 * xm * (xm_p1 - xm ** 2) - 2 * (1 - xm)
    derivative[0] = -400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0])
    derivative[-1] = 200 * (x[-1] - x[-2]**2)
    return derivative

In [4]:
def Rosenbrock_second_derivative(x):
    x = np.asarray(x)
    H = np.diag(-400 * x[:-1], 1) - np.diag(400 * x[:-1], -1)
    diagonal = np.zeros_like(x)
    diagonal[0] = 1200 * x[0]**2 - 400 * x[1] + 2
    diagonal[-1] = 200
    diagonal[1:-1] = 202 + 1200 * x[1:-1]**2 - 400 * x[2:]
    H = H + np.diag(diagonal)
    return H

In [5]:
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
res = opt.minimize(Rosenbrock, x0, method = "Newton-CG",
                   jac = Rosenbrock_derivative, hess = Rosenbrock_second_derivative,
                   options = {"xtol": 1e-8, "disp": True})
print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 24
         Function evaluations: 33
         Gradient evaluations: 33
         Hessian evaluations: 24
[1.         1.         1.         0.99999999 0.99999999]


For larger minimization problems, storing the entire Hessian matrix can consume considerable time and memory. The Newton-CG algorithm only needs the product of the Hessian times an arbitrary vector. As a result, the user can supply code to compute this product rather than the full Hessian by giving a ```hess``` function which take the minimization vector as the first argument and the arbitrary vector as the second argument (along with extra arguments passed to the function to be minimized). If possible, using Newton-CG with the Hessian product option is probably the fastest way to minimize the function.

In this case, the product of the Rosenbrock Hessian with an arbitrary vector is not difficult to compute. If 
$\mathbf{p}$ is the arbitrary vector, then $\mathbf{H}(\mathbf{x})\mathbf{p}$ has elements:

Code which makes use of this Hessian product to minimize the Rosenbrock function using minimize follows:

In [6]:
def Rosenbrock_second_derivative_p(x, p):
    x = np.asarray(x)
    Hp = np.zeros_like(x)
    Hp[0] = (1200 * x[0]**2 - 400 * x[1] + 2) * p[0] - 400 * x[0] * p[1]
    Hp[1:-1] = -400 * x[:-2] * p[:-2] + (202 + 1200 * x[1:-1]**2 - 400 * x[2:]) * p[1:-1] -400 * x[1:-1] * p[2:]
    Hp[-1] = -400 * x[-2] * p[-2] + 200 * p[-1]
    return Hp

In [7]:
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
res = opt.minimize(Rosenbrock, x0, method = "Newton-CG",
                   jac = Rosenbrock_derivative, hessp = Rosenbrock_second_derivative_p,
                   options = {"xtol": 1e-8, "disp": True})
print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 24
         Function evaluations: 33
         Gradient evaluations: 33
         Hessian evaluations: 66
[1.         1.         1.         0.99999999 0.99999999]
